## Download the data

In [ ]:
!wget https://michaeltpublic.s3.amazonaws.com/sarnet.zip

--2025-08-03 05:36:35--  https://michaeltpublic.s3.amazonaws.com/sarnet.zip
Resolving michaeltpublic.s3.amazonaws.com (michaeltpublic.s3.amazonaws.com)... 54.231.160.97, 52.217.104.68, 3.5.25.61, ...
Connecting to michaeltpublic.s3.amazonaws.com (michaeltpublic.s3.amazonaws.com)|54.231.160.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 937789201 (894M) [application/zip]
Saving to: ‘sarnet.zip’

sarnet.zip          100%[===================>] 894.34M  15.9MB/s    in 2m 23s  

2025-08-03 05:39:00 (6.24 MB/s) - ‘sarnet.zip’ saved [937789201/937789201]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q -o sarnet.zip

## Install Dependencies

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 14.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15900, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 15900 (delta 69), reused 29 (delta 29), pack-reused 15787 (from 2)
Receiving objects: 100% (15900/15900), 6.45 MiB | 18.77 MiB/s, done.
Resolving deltas: 100% (11568/11568), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your 

# Setup Data Loaders

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.engine import DefaultTrainer

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

from detectron2.data.datasets import register_coco_instances
try:
  register_coco_instances("dataset_train_1", {}, "/content/dataset/annotations/train.json", "/content/dataset/train")
  register_coco_instances("dataset_val_1", {}, "/content/dataset/annotations/val.json", "/content/dataset/val")
  register_coco_instances("dataset_test_1", {}, "/content/dataset/annotations/test.json", "/content/dataset/test")
except AssertionError:
  print("INFO: Datasets already registered.")

para_metadata = MetadataCatalog.get("dataset_train_1")
dataset_dicts = DatasetCatalog.get("dataset_train_1")

para_metadata_val = MetadataCatalog.get("dataset_val_1")
dataset_dicts_val = DatasetCatalog.get("dataset_val_1")

para_metadata_test = MetadataCatalog.get("dataset_test_1")
dataset_dicts_test = DatasetCatalog.get("dataset_test_1")





from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime
import logging
import numpy as np

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader

    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)

        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):
            if np.random.rand() > 0.1:
              continue
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            loss_batch = self._get_loss(inputs)

            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        print("validation loss on 10% of val data: {}".format(mean_loss))
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses

    def _get_loss(self, data):
        # How loss is calculated on train_loop
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced


    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD_LOSS,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks



cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = "faster_rcnn_R_50_FPN_3x.yaml"
cfg.DATASETS.TRAIN = ("dataset_train_1",)
cfg.DATASETS.TEST = ("dataset_val_1",)
cfg.TEST.EVAL_PERIOD = 1000  # do a full coco evaluation every 500 iterations
cfg.TEST.EVAL_PERIOD_LOSS = 500 # calculate the validation loss every 100 iterations
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.4]




# scaling up input images for training and testing:
cfg.INPUT.MIN_SIZE_TRAIN = (1000, 1100, 1200)
cfg.INPUT.MAX_SIZE_TRAIN = 1600
cfg.INPUT.MIN_SIZE_TEST = 1200
cfg.INPUT.MAX_SIZE_TEST = 1600





os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[08/03 05:59:32 d2.data.datasets.coco]: Loaded 1808 images in COCO format from /content/dataset/annotations/train.json
[08/03 05:59:32 d2.data.datasets.coco]: Loaded 490 images in COCO format from /content/dataset/annotations/val.json
[08/03 05:59:32 d2.data.datasets.coco]: Loaded 254 images in COCO format from /content/dataset/annotations/test.json
[08/03 05:59:33 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5)

model_final_280758.pkl: 167MB [00:00, 230MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[08/03 05:59:34 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[08/03 05:59:50 d2.utils.events]:  eta: 0:50:27  iter: 19  total_loss: 2.683  loss_cls: 0.5315  loss_box_reg: 0.001038  loss_rpn_cls: 1.932  loss_rpn_loc: 0.1349    time: 0.6215  last_time: 0.6389  data_time: 0.0491  last_data_time: 0.0517   lr: 1.9981e-06  max_mem: 3069M
[08/03 06:00:07 d2.utils.events]:  eta: 0:51:10  iter: 39  total_loss: 2.045  loss_cls: 0.4801  loss_box_reg: 0.00144  loss_rpn_cls: 1.38  loss_rpn_loc: 0.1854    time: 0.6209  last_time: 0.6520  data_time: 0.0394  last_data_time: 0.0479   lr: 3.9961e-06  max_mem: 3069M
[08/03 06:00:19 d2.utils.events]:  eta: 0:51:09  iter: 59  total_loss: 2.136  loss_cls: 0.3918  loss_box_reg: 0.0006214  loss_rpn_cls: 1.471  loss_rpn_loc: 0.1924    time: 0.6201  last_time: 0.6535  data_time: 0.0349  last_data_time: 0.0483   lr: 5.9941e-06  max_mem: 3069M
[08/03 06:00:31 d2.utils.events]:  eta: 0:51:27  iter: 79  total_loss: 1.484  loss_cls: 0.3311  loss_box_reg: 0.0008048  loss_rpn_cls: 0.9553  loss_rpn_loc: 0.1744    time: 0.6201  l